In [ ]:
# Install required libraries
!pip install transformers datasets gdown

In [ ]:
# Clone your GitHub repository for the emotion recognition project
!git clone https://github.com/Vincent-ZHQ/CA-MSER.git

# Change directory to the cloned repository
!cd CA-MSER

In [ ]:
!pip install rust
!pip install tokenizers

In [ ]:
!pip install --upgrade pip setuptools wheel

In [ ]:
!pip install setuptools-rust

In [ ]:
!pip install tokenizers

In [ ]:
pip install transformers  # Change the version as needed

In [ ]:
!pip install torch

In [ ]:
pip install transformers

In [1]:
# Download Wav2Vec2.0 pretrained model from Hugging Face
from transformers import Wav2Vec2Model

In [ ]:
!pip install imbalanced-learn
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [1]:
import torch
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
features_file = '/content/drive/MyDrive/path/to/reduced_file.pkl'

Is CUDA available: True
CUDA version: 11.8


In [2]:
import sys
import os
import time
import json  # Import json for saving results

# Add the path to the directory containing train_ser.py to the Python path
sys.path.append('CA-MSER')  # Change the path if train_ser.py is in a different directory

import train_ser
from train_ser import parse_arguments
import sys
import pickle
import os
import time


repeat_kfold = 2 # to  perform 10-fold for n-times with different seed
localtime = time.localtime(time.time())
str_time = f'{str(localtime.tm_year)}-{str(localtime.tm_mon)}-{str(localtime.tm_mday)}-{str(localtime.tm_hour)}-{str(localtime.tm_min)}'

#------------PARAMETERS---------------#

features_file = 'IEMOCAP_multi.pkl'

#  leave-one-speaker-out
# val_id = ['1M','1F','2M','2F','3M','3F','4M','4F','5M','5F']
# test_id = ['1M','1F','2M','2F','3M','3F','4M','4F','5M','5F']

#  leave-one-session-out
val_id = ['1M','2M','3M','4M','5M']
test_id = ['1F','2F','3F','4F','5F']

num_epochs  = '50'
early_stop = '10'
batch_size  = '32'
lr          = '0.00001'
random_seed = 111
gpu = '1'
gpu_ids = ['0']
save_label = str_time#'0930_01'#'alexnet_pm_0704'


#Start Cross Validation
all_stat = []

for repeat in range(repeat_kfold):

    random_seed +=  (repeat*100)
    seed = str(random_seed)

    for v_id, t_id in list(zip(val_id, test_id)):

        train_ser.sys.argv      = [

                                  'train_ser.py',
                                  features_file,
                                  '--repeat_idx', str(repeat),
                                  '--val_id',v_id,
                                  '--test_id', t_id,
                                  '--gpu', gpu,
                                  '--gpu_ids', gpu_ids,
                                  '--num_epochs', num_epochs,
                                  '--early_stop', early_stop,
                                  '--batch_size', batch_size,
                                  '--lr', lr,
                                  '--seed', seed,
                                  '--save_label', save_label,#,
                                  '--pretrained'
                                  ]


        stat = train_ser.main(parse_arguments(sys.argv[1:]))
        all_stat.append(stat)
        os.remove(save_label+'.pth')

    # with open('allstat_iemocap_'+save_label+'_'+str(repeat)+'.pkl', "wb") as fout:
    #     pickle.dump(all_stat, fout)

n_total = repeat_kfold*len(val_id)
total_best_epoch, total_epoch, total_loss, total_wa, total_ua = 0, 0, 0, 0, 0

for i in range(n_total):
    print(i, ": ", all_stat[i][0], all_stat[i][1], all_stat[i][8], all_stat[i][9], all_stat[i][10])
    total_best_epoch += all_stat[i][0]
    total_epoch += all_stat[i][1]
    total_loss += float(all_stat[i][8])
    total_wa += float(all_stat[i][9])
    total_ua += float(all_stat[i][10])

print("AVERAGE:", total_best_epoch/n_total, total_epoch/n_total, total_loss/n_total, total_wa/n_total, total_ua/n_total )

print(all_stat)

****************************************

PARAMETERS:

****************************************


       ser_task: SLM
     repeat_idx: 0
         val_id: 1M
        test_id: 1F
     num_epochs: 50
     early_stop: 10
     batch_size: 32
             lr: 1e-05
    random_seed: 111
        use_gpu: 1
        gpu_ids: ['0']
     save_label: 2024-10-20-15-22
   oversampling: False
     pretrained: True
****************************************



Dataset normalized with minmax scaler
	Range before normalization: [-80.0, 3.8146973e-06]
	Range after  normalization: [0.0, 1.0]

<<DATASET>>

Val. speaker id : 1M
Test speaker id : 1F
Train data      : (8778, 1, 200, 300)
Train labels    : (8778,)
Eval. data      : (1187, 1, 200, 300)
Eval. label     : (557,)
Eval. seg labels: (1187,)
Eval. num seg   : (557,)
Test data       : (1037, 1, 200, 300)
Test label      : (528,)
Test seg labels : (1037,)
Test num seg    : (528,)


pytorch version:  2.1.1+cu118
cuda version:  11.8
cudnn version:  8700
gp

/home/stud1/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/stud1/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



<< SER AlexNet Finetuning model initialized >>



/home/stud1/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ser_Model(
  (alexnet_model): SER_AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=Fa

...

 ang  sad  hap  neu
  98    1   28   21
   2  100    9   22
   5   17  122   14
  10   33   35  112
26 30
Epoch 31  (lr = 1e-05)        Loss: 0.2460 - 1.2974 - WA: 68.68% <71.54%> - UA: 69.39% <71.92%>


100%|██████████| 20/20 [00:03<00:00,  6.18it/s]


 ang  sad  hap  neu
 111    1   18   18
   3  109    8   13
  15   20  109   14
  16   34   25  115
26 31
Epoch 32  (lr = 1e-05)        Loss: 0.2281 - 1.3558 - WA: 70.59% <71.54%> - UA: 71.62% <71.92%>


 86%|████████▌ | 235/274 [00:57<00:09,  4.12it/s]


KeyboardInterrupt: 

In [1]:
import numpy as np

# Confusion matrices
matrix_1 = np.array([
    [112, 2, 9, 24],
    [1, 62, 8, 7],
    [13, 7, 95, 17],
    [5, 49, 27, 90]
])

matrix_2 = np.array([
    [56, 1, 4, 6],
    [2, 75, 23, 13],
    [4, 5, 142, 15],
    [5, 17, 37, 76]
])

# Normalizing the rows by dividing each element by the row sum
def normalize(matrix):
    return matrix / matrix.sum(axis=1, keepdims=True)

# Normalize both matrices
normalized_matrix_1 = normalize(matrix_1)
normalized_matrix_2 = normalize(matrix_2)

# Compute the element-wise average of the two normalized matrices
averaged_matrix = (normalized_matrix_1 + normalized_matrix_2) / 2

# Output the results
print("Normalized Matrix 1:")
print(normalized_matrix_1)

print("\nNormalized Matrix 2:")
print(normalized_matrix_2)

print("\nAveraged Confusion Matrix:")
print(averaged_matrix)

Normalized Matrix 1:
[[0.76190476 0.01360544 0.06122449 0.16326531]
 [0.01282051 0.79487179 0.1025641  0.08974359]
 [0.09848485 0.0530303  0.71969697 0.12878788]
 [0.02923977 0.28654971 0.15789474 0.52631579]]

Normalized Matrix 2:
[[0.8358209  0.01492537 0.05970149 0.08955224]
 [0.01769912 0.66371681 0.20353982 0.11504425]
 [0.02409639 0.03012048 0.85542169 0.09036145]
 [0.03703704 0.12592593 0.27407407 0.56296296]]

Averaged Confusion Matrix:
[[0.79886283 0.01426541 0.06046299 0.12640877]
 [0.01525981 0.7292943  0.15305196 0.10239392]
 [0.06129062 0.04157539 0.78755933 0.10957466]
 [0.0331384  0.20623782 0.21598441 0.54463938]]
